In [1]:
from TorchMetricLogger import TorchMetricLogger as TML
from TorchMetricLogger import TmlMetric, TmlMetricFunction, TMLBinaryAccuracy, TMLDiceCoefficient, TMLMean
import torch
import numpy as np

In [2]:
loss = np.linspace(2.0, 0.0, num=50)

In [3]:
tml = TML()

for l in loss:
    tml(loss=TmlMetric(None, l, TMLMean))

In [4]:
tml.on_batch_end()

In [13]:
np.allclose(tml.metrics["loss"].history, 1)

True

In [6]:
import numpy as np
a = np.array(tml.metrics["loss"].partial)

In [7]:
a.shape

(0,)

In [8]:
tml.on_batch_end()
tml.metrics["train_bin_accuracy"].history

IndexError: too many indices for array

In [ ]:
from TorchMetricLogger.torchmetricfunction import TMLBinaryAccuracy